### Import librairies

In [1]:
import numpy as np
import os
from PIL import Image
from collections import defaultdict
from itertools import product
from sklearn.model_selection import train_test_split
import shutil
import re
import glob
from scipy import ndimage
import pickle
from six.moves import cPickle as pickle
from six.moves import range
from __future__ import division, print_function, absolute_import
import tflearn
from tflearn.data_utils import shuffle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
import tensorflow as tf


## Load Data 

I am using Flick-27 logo dataset which is publicly available and has images download from Flickr of 27 brands. It has 30 images for each class. In total 810 images for training. Although it is a very small dataset for Deep Learning problem so I will use Data Augmentation techniques that can inflated the data to a bigger dataset suitable for training an object detection model.

In [2]:
width = 32
height = 32

posshiftshift_min = -5
posshiftshift_max = 5
scales = [0.9, 1.1]
rot_min = -15
rot_max = 15

dir = 'flickrData'
imgdir = os.path.join(dir, 'flickr_logos_27_dataset_images')
pp_dir = os.path.join(
    dir, 'processedF')
annot = 'flickr_logos_27_dataset_training_set_annotation.txt'


In [12]:
annot_train = np.loadtxt(os.path.join(dir, annot), dtype='a')
print('train_annotation: %d, %d ' % (annot_train.shape))       

train_annotation: 4536, 7 


## Crop and augmente data 

#### Useful function for the augmentation 

In [3]:
def parse_annot(annot):
    fn = annot[0].decode('utf-8')
    class_name = annot[1].decode('utf-8')
    train_subset_class = annot[2].decode('utf-8')
    return fn, class_name, train_subset_class

# extarcting the x and y coordinates of the logo given in the txt file
def get_rect(annot):             
    rect = defaultdict(int)
    x1, y1, x2, y2 = rect_coord(annot[3:])
    cx, cy, wid, hgt = center_wid_hgt(x1, y1, x2, y2)
    rect['x1'] = x1
    rect['y1'] = y1
    rect['x2'] = x2
    rect['y2'] = y2
    rect['cx'] = cx
    rect['cy'] = cy
    rect['wid'] = wid
    rect['hgt'] = hgt
    return rect

In [4]:
# Shifting position of the extracted logo.
def posshift(annot, im):   
    posshift_ims = []
    posshift_suffixes = []

    rect = get_rect(annot)
    for sx, sy in product(                        
            range(posshiftshift_min, posshiftshift_max),
            range(posshiftshift_min, posshiftshift_max)):
        cx = rect['cx'] + sx
        cy = rect['cy'] + sy
        cropped_im = im.crop((cx - rect['wid'] // 2, cy - rect['hgt'] // 2,
                              cx + rect['wid'] // 2, cy + rect['hgt'] // 2))
        resized_im = cropped_im.resize((width, height))
        posshift_ims.append(resized_im)
        posshift_suffixes.append('p' + str(sx) + str(sy))
        cropped_im.close()

    return posshift_ims, posshift_suffixes

In [5]:
# For resizing the extracted logo.
def scale(annot, im):   
    scale_ims = []
    scale_suffixes = []

    rect = get_rect(annot)
    for s in scales:
        w = int(rect['wid'] * s)
        h = int(rect['hgt'] * s)
        cropped_im = im.crop((rect['cx'] - w // 2, rect['cy'] - h // 2,
                              rect['cx'] + w // 2, rect['cy'] + h // 2))
        resized_im = cropped_im.resize((width, height))
        scale_ims.append(resized_im)
        scale_suffixes.append('s' + str(s))
        cropped_im.close()

    return scale_ims, scale_suffixes



In [6]:
# Rotating the extracted logo.
def rotate(annot, im):   
    rotate_ims = []
    rotate_suffixes = []

    rect = get_rect(annot)
    for r in range(rot_min, rot_max):
        rotated_im = im.rotate(r)
        cropped_im = rotated_im.crop(
            (rect['cx'] - rect['wid'] // 2, rect['cy'] - rect['hgt'] // 2,
             rect['cx'] + rect['wid'] // 2, rect['cy'] + rect['hgt'] // 2))
        resized_im = cropped_im.resize((width, height))
        rotate_ims.append(resized_im)
        rotate_suffixes.append('r' + str(r))
        rotated_im.close()
        cropped_im.close()

    return rotate_ims, rotate_suffixes

#### Useful function for cropping 

In [7]:
#Cropping the logo
def crop(annot, im):                        
    x1, y1, x2, y2 = rect_coord(annot[3:])
    cropped_im = im.crop((x1, y1, x2, y2))
    cropped_im = cropped_im.resize((width, height))
    cropped_suffix = 'p00'
    return [cropped_im], [cropped_suffix]

# apply on each list item and collect all the return values.
def rect_coord(annot_part):
    return list(map(int, annot_part))   


def center_wid_hgt(x1, y1, x2, y2):
    cx = x1 + (x2 - x1) // 2
    cy = y1 + (y2 - y1) // 2
    wid = (x2 - x1)
    hgt = (y2 - y1)
    return cx, cy, wid, hgt

In [8]:
def is_skip(annot_part):
    x1, y1, x2, y2 = rect_coord(annot_part)
    _, _, wid, hgt = center_wid_hgt(x1, y1, x2, y2)
    if wid <= 0 or hgt <= 0:
        return True
    else:
        return False

# Saving the processed image.
def save_im(annot, cnt, *args):         
    fn, class_name, train_subset_class = parse_annot(annot)
    dst_dir = os.path.join(pp_dir, class_name)
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
    for i, arg in enumerate(args):
        for im, suffix in zip(arg[0], arg[1]):
            save_fn = '_'.join([
                fn.split('.')[0], class_name, train_subset_class, str(cnt),
                suffix
            ]) + os.path.splitext(fn)[1]
            im.save(os.path.join(dst_dir, save_fn))


def close_im(*args):
    for ims in args:
        for im in ims:
            im.close()


In [9]:
 # parent function which call all the sub functions.
def crop_and_aug(annot_train):    
    cnt_per_file = defaultdict(int)
    for annot in annot_train:
        # for generating a file name
        fn, _, _ = parse_annot(annot)
        cnt_per_file[fn] += 1

        # skip if width or height equal zero
        if is_skip(annot[3:]):
            print('Skip: ', fn)
            continue

        # open an image
        im = Image.open(os.path.join(imgdir, fn))

        # normal cropping
        cropped_ims, cropped_suffixes = crop(annot, im)

        # augment by shifting a center
        shifted_ims, shifted_suffixes = posshift(annot, im)

        # augment by scaling
        scaled_ims, scaled_suffixes = scale(annot, im)

        # augment by rotation
        rotated_ims, rotated_suffixes = rotate(annot, im)

        # save images
        save_im(annot, cnt_per_file[fn], [cropped_ims, cropped_suffixes],
                [shifted_ims, shifted_suffixes], [scaled_ims, scaled_suffixes],
                [rotated_ims, rotated_suffixes])

        # close image file
        close_im([im], cropped_ims, shifted_ims, scaled_ims, rotated_ims)

In [10]:
def crop_and_aug_with_none(annot_train, with_none=False):
    # root directory to save processed images
    if not os.path.exists(pp_dir):
        os.makedirs(pp_dir)

    # crop images and apply augmentation
    crop_and_aug(annot_train)

    # print results
    org_imgs = [img for img in os.listdir(imgdir)]
    crop_and_aug_imgs = [
        fname
        for root, dirs, files in os.walk(pp_dir)
        for fname in glob.glob(os.path.join(root, '*.jpg'))  # look for the file with .jpg extension.
    ]
    print('original: %d' % (len(org_imgs)))
    print('cropped: %d' % (len(crop_and_aug_imgs)))  

#### Useful function for  splitting data

In [11]:
# For spliting dataset into 2 directories Train and Test.
def do_train_test_split():                                   
    class_names = [cls for cls in os.listdir(pp_dir)]
   # create directories under a particular class name.
    for class_name in class_names:                           
        if os.path.exists(                                  
                os.path.join(pp_dir, class_name, 'train')):
            continue
        if os.path.exists(
                os.path.join(pp_dir, class_name, 'test')):
            continue

        imgs = [
            img
            for img in os.listdir(
                os.path.join(pp_dir, class_name))
        ]
        # train=0.75, test=0.25
        train_imgs, test_imgs = train_test_split(imgs)
        # move images to train or test directory
        # create directories
        os.makedirs(os.path.join(pp_dir, class_name, 'train'))         
        os.makedirs(os.path.join(pp_dir, class_name, 'test'))
        for img in train_imgs:
            dst = os.path.join(pp_dir, class_name, 'train')
            src = os.path.join(pp_dir, class_name, img)
            # moving image into that directory
            shutil.move(src, dst)                                      
        for img in test_imgs:
            dst = os.path.join(pp_dir, class_name, 'test')
            src = os.path.join(pp_dir, class_name, img)
            shutil.move(src, dst)                                     


#### Implementation 

In [26]:
# cropping and data augmentation
crop_and_aug_with_none(annot_train)
# train_test_split
do_train_test_split()


Skip:  2662264721.jpg
Skip:  2662264721.jpg
Skip:  2662264721.jpg
Skip:  2662264721.jpg
Skip:  2662264721.jpg
original: 1079
cropped: 598092


## Pickle File 

In [27]:
#Parameters of the image
width = 32     
height = 32    
channel = 3         
pix_val = 255.0

dir = 'flickrData'
# Directory where processed images are stored
pp_dir = os.path.join(dir, 'processedF') 
# Name of the pickle file
pickle_file = 'logo_dataset.pickle'    
# Number of images stored as training dataset in pickle file from the processed images
train_size = 70000  
val_size = 5000
# Number of images stored as test dataset in pickle file from the processed images
test_size = 7000    

#### Useful functions 

In [30]:
# Creates array of dataset

def array(nb_rows, image_width, image_height, image_ch=1):     
    if nb_rows:
        dataset = np.ndarray(                               #  stores its height, width and channel into an array
            (nb_rows, image_height, image_width, image_ch), dtype=np.float32)
        labels = np.ndarray(nb_rows, dtype=np.int32)        #  stores its labels
    else:
        dataset, labels = None, None
    return dataset, labels

In [31]:
# Merging pickle files of all the classes into one pickle file.
def combine(pickle_files, train_size, val_size=0):     
    num_classes = len(pickle_files)
    valid_dataset, valid_labels = array(val_size, width,
                                              height, channel)
    train_dataset, train_labels = array(train_size, width,
                                              height, channel)
    vsize_per_class = val_size // num_classes
    tsize_per_class = train_size // num_classes

    start_v, start_t = 0, 0
    end_v, end_t = vsize_per_class, tsize_per_class
    end_l = vsize_per_class + tsize_per_class
    for label, pickle_file in enumerate(pickle_files):
        try:
            with open(pickle_file, 'rb') as f:
                logo_set = pickle.load(f)
                np.random.shuffle(logo_set)
                if valid_dataset is not None:
                    valid_logo = logo_set[:vsize_per_class, :, :, :]
                    valid_dataset[start_v:end_v, :, :, :] = valid_logo
                    valid_labels[start_v:end_v] = label
                    start_v += vsize_per_class
                    end_v += vsize_per_class
                train_logo = logo_set[vsize_per_class:end_l, :, :, :]
                train_dataset[start_t:end_t, :, :, :] = train_logo
                train_labels[start_t:end_t] = label
                start_t += tsize_per_class
                end_t += tsize_per_class
        except Exception as e:
            print('Unable to process data from', pickle_file, ':', e)
            raise
    return valid_dataset, valid_labels, train_dataset, train_labels



In [32]:
def makepickle(train_dataset, train_labels, valid_dataset, valid_labels,   
                test_dataset, test_labels):
    try:
        f = open(pickle_file, 'wb')
        save = {
            'train_dataset': train_dataset,
            'train_labels': train_labels,
            'valid_dataset': valid_dataset,
            'valid_labels': valid_labels,
            'test_dataset': test_dataset,
            'test_labels': test_labels,
        }
        pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)      # Saving data of the images into a pickle file
        f.close()
    except Exception as e:
        print('Unable to save data to', pickle_file, ':', e)
        raise

In [28]:
# Opening the image
def load_logo(data_dir):    
    image_files = os.listdir(data_dir)      
    dataset = np.ndarray(
        shape=(len(image_files), height, width, channel),
        dtype=np.float32)
    print(data_dir)
    num_images = 0
    for image in image_files:
        image_file = os.path.join(data_dir, image)
        try:
            image_data = (ndimage.imread(image_file).astype(float) -        
                          pix_val / 2) / pix_val
            if image_data.shape != (height, width, channel):
                raise Exception('Unexpected image shape: %s' %
                                str(image_data.shape))
            dataset[num_images, :, :] = image_data
            num_images = num_images + 1
        except IOError as e:
            print('Could not read:', image_file, ':', e,
                  '-it\'s ok, skipping.')

    dataset = dataset[0:num_images, :, :]                           
    print('Full dataset tensor:', dataset.shape)       # Tell processed number of images for a particular class 
    print('Mean:', np.mean(dataset))                   # Calculate mean over that entire class
    print('Standard deviation:', np.std(dataset))      # Calculate standard deviation over that entire class
    return dataset

In [29]:
# Creating Pickle file
def pickling(data_dirs, force=False):     
    dataset_names = []
    for dir in data_dirs:
        set_filename = dir + '.pickle'
        dataset_names.append(set_filename)

         
        if os.path.exists(set_filename) and force:    
    
            print('%s already present - Skipping pickling. ' % set_filename)
        else:
            print('Pickling %s.' % set_filename)
            dataset = load_logo(dir)
            try:
                with open(set_filename, 'wb') as f:
                    pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
            except Exception as e:
                print('Unable to save data to', set_filename, ':', e)
    return dataset_names


In [33]:
def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation, :, :]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels

#### Implementation 

In [34]:
CLASS_NAME = [
    'Apple', 'BMW','Heineken','HP','Intel','Mini','Starbucks','Vodafone', 'Citroen', 'Ferrari'
]

In [35]:
dirs = [
        os.path.join(pp_dir, class_name, 'train')      # Look into all the train folder of the class
        for class_name in CLASS_NAME
    ]
test_dirs = [
        os.path.join(pp_dir, class_name, 'test')        # Look into all the test folder of the class
        for class_name in CLASS_NAME
    ]

train_datasets = pickling(dirs)
test_datasets = pickling(test_dirs)




Pickling flickrData\processedF\Apple\train.pickle.
flickrData\processedF\Apple\train


D:\a\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  # This is added back by InteractiveShellApp.init_path()


Full dataset tensor: (25443, 32, 32, 3)
Mean: 0.116072275
Standard deviation: 0.2986908
Pickling flickrData\processedF\BMW\train.pickle.
flickrData\processedF\BMW\train
Full dataset tensor: (11781, 32, 32, 3)
Mean: -0.07783215
Standard deviation: 0.32733378
Pickling flickrData\processedF\Heineken\train.pickle.
flickrData\processedF\Heineken\train
Full dataset tensor: (15939, 32, 32, 3)
Mean: -0.08269091
Standard deviation: 0.30394986
Pickling flickrData\processedF\HP\train.pickle.
flickrData\processedF\HP\train
Full dataset tensor: (10989, 32, 32, 3)
Mean: -0.11493689
Standard deviation: 0.348335
Pickling flickrData\processedF\Intel\train.pickle.
flickrData\processedF\Intel\train
Full dataset tensor: (12177, 32, 32, 3)
Mean: 0.12077578
Standard deviation: 0.3359874
Pickling flickrData\processedF\Mini\train.pickle.
flickrData\processedF\Mini\train
Full dataset tensor: (10395, 32, 32, 3)
Mean: -0.08043128
Standard deviation: 0.31972796
Pickling flickrData\processedF\Starbucks\train.pickl

In [ ]:
valid_dataset, valid_labels, train_dataset, train_labels = combine(train_datasets, train_size, val_size)# function called for merging
test_dataset, test_labels = combine(test_datasets, test_size)

train_dataset, train_labels = randomize(train_dataset, train_labels)   # function called for randomizing
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)  
test_dataset, test_labels = randomize(test_dataset, test_labels)

In [37]:
makepickle(train_dataset, train_labels, valid_dataset, valid_labels,test_dataset, test_labels)# function called for making a pickle file.
statinfo = os.stat(pickle_file)                         # Shows size of the file 
print('Compressed pickle size:', statinfo.st_size)


Compressed pickle size: 1007944515


## Read Data

In [39]:

def read_data():
    with open("logo_dataset.pickle", 'rb') as f:
        save = pickle.load(f)
        X = save['train_dataset']       # assign X as train dataset
        Y = save['train_labels']        # assign Y as train labels 
        X_test = save['test_dataset']   # assign X_test as test dataset
        Y_test = save['test_labels']    #assign Y_test as test labels
        del save
    return [X, X_test], [Y, Y_test]

def reformat(dataset, labels):   
    dataset = dataset.reshape((-1, 32, 32,3)).astype(np.float32)    # Reformatting shape array to give a scalar value for dataset.  
    labels = (np.arange(10) == labels[:, None]).astype(np.float32) 
    return dataset, labels

dataset, labels = read_data()
X,Y = reformat(dataset[0], labels[0])
X_test, Y_test = reformat(dataset[1], labels[1])
print('Training set', X.shape, Y.shape)
print('Test set', X_test.shape, Y_test.shape)            

# Shuffle the data
X, Y = shuffle(X, Y)    # Imported from TFLearn.



Training set (70000, 32, 32, 3) (70000, 10)
Test set (7000, 32, 32, 3) (7000, 10)


## Build Model: Convolutional network

In [40]:
# Make sure the data is normalized
img_prep = ImagePreprocessing()    
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

# Create extra synthetic training data by flipping images on our data set.
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()

In [41]:

# Define the convolutional network architecture:

# 32x32 image is the input with 3 color channels (red, green and blue) with it's mean and standard deviation.
network = input_data(shape=[None, 32, 32, 3], data_preprocessing=img_prep, data_augmentation=img_aug)


# Convolution 1 with 64 nodes with activation function as rectified linear unit:
network = conv_2d(network, 64, 3, activation='relu')

# Max pooling 1:
network = max_pool_2d(network, 2)

# Convolution 2 with 128 nodes with activation function as rectified linear unit:
network = conv_2d(network, 128, 3, activation='relu')

# Convolution 3 with 256 nodes with activation function as rectified linear unit:
network = conv_2d(network, 256, 3, activation='relu')

# Max pooling 2:
network = max_pool_2d(network, 2)

# Fully-connected 512 node neural network with activation function as rectified linear unit
network = fully_connected(network, 512 , activation='relu')

# Dropout - throw away some data randomly during training to prevent over-fitting
network = dropout(network, 0.5)

# Fully-connected neural network for outputs with activation function as softmax as we are dealing with multiclass classification.
network = fully_connected(network, 10, activation='softmax')

# To train the network we will use adaptive moment estimation (ADAM) and categorical_crossentropy  
# to determine loss in learning process and optimization.
network = regression(network, optimizer='adam', loss='categorical_crossentropy', learning_rate=0.001)

# Covering the network in a model object
model = tflearn.DNN(network, tensorboard_verbose=0, checkpoint_path="model\logo-classifier.tfl.ckpt")



Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


### Train model

In [42]:
# Training! n_epoch will tell how many iterations the network has to go through, here it is kept 15 training passes and monitor it as it goes.
model.fit(X,Y, n_epoch=10, shuffle=True, validation_set=(X_test, Y_test), show_metric=True, batch_size=128, snapshot_epoch=True,
          run_id='logo-classifier')

# Save model when training is complete to a file
model.save("logo-classifier.tfl")
print("Network trained and saved as logo-classifier.tfl!")

Training Step: 5469  | total loss: 0.23238 | time: 2388.690s
| Adam | epoch: 010 | loss: 0.23238 - acc: 0.9842 -- iter: 69888/70000
Training Step: 5470  | total loss: 0.20941 | time: 2466.489s
| Adam | epoch: 010 | loss: 0.20941 - acc: 0.9858 | val_loss: 0.01089 - val_acc: 0.9964 -- iter: 70000/70000
--
INFO:tensorflow:C:\Users\aMaL\Desktop\logo\model\logo-classifier.tfl.ckpt-5470 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:C:\Users\aMaL\Desktop\logo\logo-classifier.tfl is not in all_model_checkpoint_paths. Manually adding it.
Network trained and saved as logo-classifier.tfl!


## Test the model 

In [59]:
#Loading the trained dataset file 'logo-classifier.tfl.ckpt-4923'
model.load("model\logo-classifier.tfl.ckpt-4923")



INFO:tensorflow:Restoring parameters from C:\Users\aMaL\Desktop\logo\model\logo-classifier.tfl.ckpt-4923


In [61]:
#Evaluate the model
score=model.evaluate(X_test, Y_test)
print(score)

[0.9964285714285714]
